---

### Комментарий наставника

Привет!

Спасибо за проделанную работу!

Все мои комментарии ты увидишь ниже.

---

# SQL - запрос для маркетплейсе «Заберу»

Вы аналитик в онлайн-маркетплейсе «Заберу». Это настоящий рай для любителей бесполезных сувениров — там можно приобрести товары ручной работы от небольших производителей. «Заберу» проводит рекламные акции длительностью 1 день. Руководитель отдела маркетинга хочет узнать, насколько эффективны такие кампании.

In [121]:
# импорт необходимых библиотек

import pandas as pd
from sqlalchemy import create_engine
import plotly.express as px

In [122]:
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                            db_config['pwd'],
                                                            db_config['host'],
                                                            db_config['port'],
                                                            db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'}, echo=True)

#### Описание таблиц

Таблица visits (лог сервера с информацией о посещениях сайта):
- uid — уникальный идентификатор пользователя
- device — категория устройства пользователя
- startts — дата и время начала сессии
- endts — дата и время окончания сессии
- sourceid — идентификатор рекламного источника, из которого пришёл пользователь


Таблица orders (информация о заказах):
- uid — уникальный идентификатор пользователя, который сделал заказ
- buyts — дата и время заказа
- revenue — выручка с этого заказа


Таблица advertisment_costs (информация о затратах на маркетинг):
- sourceid — идентификатор рекламного источника
- dt — дата
- costs — затраты на этот рекламный источник в этот день

### Посчитаем затраты на каждый рекламный источник.

In [123]:
query1 = '''
            SELECT 
                sourceid,
                SUM(costs) AS costs_sum
            FROM 
                advertisment_costs
            GROUP BY 
                sourceid
        '''

In [124]:
query1_exit = pd.io.sql.read_sql(query1, con = engine)

2020-03-17 22:01:56,588 INFO sqlalchemy.engine.base.Engine select version()
2020-03-17 22:01:56,588 INFO sqlalchemy.engine.base.Engine {}
2020-03-17 22:01:56,654 INFO sqlalchemy.engine.base.Engine select current_schema()
2020-03-17 22:01:56,658 INFO sqlalchemy.engine.base.Engine {}
2020-03-17 22:01:56,714 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-03-17 22:01:56,718 INFO sqlalchemy.engine.base.Engine {}
2020-03-17 22:01:56,750 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-03-17 22:01:56,754 INFO sqlalchemy.engine.base.Engine {}
2020-03-17 22:01:56,778 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2020-03-17 22:01:56,782 INFO sqlalchemy.engine.base.Engine {}
2020-03-17 22:01:56,842 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

In [119]:
query1_exit

,sourceid,costs_sum
0,10,"$5,822.49"
1,9,"$5,517.49"
2,1,"$20,833.27"
3,5,"$51,757.10"
4,4,"$61,073.60"
5,2,"$42,806.04"
6,3,"$141,321.63"


In [120]:
# визуализируем данных в графике
fig_street = px.bar(query1_exit, x='sourceid', y='costs_sum', title='Распределение расходов по рекламным каналам')
fig_street.update_xaxes(tickangle=30)
fig_street.show()

##### Вывод

Больше всего затрачено на маркетинговый канал под номером №3. Затраты составляют 141 321.63 долларов.

Меньше всего выделено средств на рекламный канал №9 - 5 517.49 долларов.

---

### Комментарий наставника

Согласен с тобой! Запрос написан верно

---

### Посчитаем суммарную выручку, максимальную и минимальную выручку от заказов, сделанных с настольных ПК (тип устройства desktop) пользователями, которые хотя бы раз перешли из рекламного источника, на который было потрачено больше всего денег. 

In [102]:
query2 = '''
        SELECT 
            SUM(orders.revenue), 
            MIN(orders.revenue), 
            MAX(orders.revenue)
        FROM 
            orders
        WHERE
            orders.uid IN (
                SELECT 
                    DISTINCT visits.uid
                FROM 
                    visits
                WHERE
                    visits.device = 'desktop' AND visits.sourceid = '3');
            '''

In [103]:
query2_exit = pd.io.sql.read_sql(query2, con = engine)

2020-03-17 21:48:14,421 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-17 21:48:14,425 INFO sqlalchemy.engine.base.Engine {'name': "\n        SELECT \n            SUM(orders.revenue), \n            MIN(orders.revenue), \n            MAX(orders.revenue)\n        FROM \n             ... (130 characters truncated) ...     FROM \n                    visits\n                WHERE\n                    visits.device = 'desktop' AND visits.sourceid = '3');\n            "}
2020-03-17 21:48:14,491 INFO sqlalchemy.engine.base.Engine 
        SELECT 
            SUM(orders.revenue), 
            MIN(orders.revenue), 
            MAX(orders.revenue)
        FROM 
            orders
        WHERE
            orders.uid IN (
                SELECT 
                    DISTINCT visits.uid
                FROM 
                    visits
                WHERE
         

In [104]:
query2_exit

,sum,min,max
0,"$106,058.86",$0.00,"$2,633.28"


##### Вывод 

Наименьший заказ, совершенный через 3 (самый прибыльный) рекламный канал, составил всего 0 долларов - возможно это итоговая цена после применений акций/скидок.

Самая дорогая покупка - 2633,28 долларов. Красиво жить не запретишь.

Суммарная прибыль 106058,86 долларов. Хорошо бы оценить еще доход с мобильных устройств, чтобы проверить окупились ли наши затраты на данный канал связи.

---

### Комментарий наставника

Да, это действительно так!

---

### Введём новую метрику эффективности рекламы - дневные затраты на рекламу на пользователя по источнику. Она рассчитывается для каждого отдельного дня и равна объёму денежных средств, которые были израсходованы на то, чтобы пользователь перешёл по рекламе на сайт из данного рекламного источника. Иными словами, метрика для каждого дня равна отношению рекламных затрат по данному источнику к количеству посетителей, перешедших на сайт с данного источника. Рассчитайте для каждого дня введённую метрику. 

In [105]:
query3 = '''
        SELECT
            final_data.day,
            final_data.sourceid,
            advertisment_costs.costs / final_data.count_users as per_user_cost
        FROM
        advertisment_costs JOIN
        (SELECT
                DATE_TRUNC('day', startts::timestamp) as day,
                count(uid) as count_users,
                sourceid
            FROM
                visits
            GROUP BY
                sourceid,
                day
        ) as final_data ON advertisment_costs.sourceid = final_data.sourceid AND 
        DATE_TRUNC('day', advertisment_costs.dt::timestamp) = final_data.day
        ORDER BY
            final_data.day,
            final_data.sourceid;
    
            '''

In [106]:
query3_exit = pd.io.sql.read_sql(query3, con = engine)

2020-03-17 21:48:14,986 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-17 21:48:14,990 INFO sqlalchemy.engine.base.Engine {'name': "\n        SELECT\n            final_data.day,\n            final_data.sourceid,\n            advertisment_costs.costs / final_data.count_users as per ... (432 characters truncated) ... vertisment_costs.dt::timestamp) = final_data.day\n        ORDER BY\n            final_data.day,\n            final_data.sourceid;\n    \n            "}
2020-03-17 21:48:15,058 INFO sqlalchemy.engine.base.Engine 
        SELECT
            final_data.day,
            final_data.sourceid,
            advertisment_costs.costs / final_data.count_users as per_user_cost
        FROM
        advertisment_costs JOIN
        (SELECT
                DATE_TRUNC('day', startts::timestamp) as day,
                count(uid) as count_users,
              

In [107]:
query3_exit

,day,sourceid,per_user_cost
0,2017-06-01,1,$1.10
1,2017-06-01,2,$1.55
2,2017-06-01,3,$1.88
3,2017-06-01,4,$0.54
4,2017-06-01,5,$0.60
...,...,...,...
2536,2018-05-31,3,$2.03
2537,2018-05-31,4,$0.96
2538,2018-05-31,5,$1.57
2539,2018-05-31,9,$0.44


##### Преобразования с помощью pandas

In [108]:
# переведем столбец с долларами к типу числа с точкой, чтобы сделать выводы по полученной информации
query3_exit['per_user_cost'] = query3_exit['per_user_cost'].str.replace('$', '').astype('float64')

In [116]:
# построим сводную таблицу и определеим какой канал рекламы самый затратный 
(query3_exit
 .pivot_table(index='sourceid', values='per_user_cost')
 .reset_index()
 .sort_values(by='per_user_cost', ascending=False))

,sourceid,per_user_cost
2,3,1.682039
5,9,0.979284
1,2,0.940386
4,5,0.786364
6,10,0.783471
3,4,0.669477
0,1,0.654105


##### Вывод

Подготовив запрос и выгрузив информацию, можно посмотреть информацию о самом затратном рекламном канале.

Как показывает сводная таблица, на третий канал связи потрачено больше всего средств, при этом он же и самый затратный в соотношении рекламный затраты на число переходов по нему. Необходимо либо повышать вовлеченность, либо снижать затраты. 

Лучший показатель - у рекламного канала №1 и №4, при меньших расходах показывает больший процент перехода. Стоит обратить на них внимание при дальнейшем планировании работы.

---

### Комментарий наставника

Работа проделана чётко! Класс :)

Принимаю. Желаю успеха в работе над проектом!

---